<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_single_ligand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick Instructions

## Choose your runtime:

If you want to use CUDA, a GPU computing API that increases the speed in certain operations, you need to check the ```use_cuda``` box and select the GPU runtime from Google Colab by following the steps in the menu:

> ### `Runtime > Change Runtime Type > T4 GPU`


In case you don't want to use (or don't have acess to) a GPU, pyLiBELa will also work smoothly, but the Grid generation will take longer.

## Required files:

To run pyLiBELa in this notebook, you will need:
* A receptor file in MOL2 format.
* A reference ligand in MOL2 format.
* A ligand to be docked in SMILES.

Make sure that the MOL2 files include hydrogen atoms and atomic charges. You can add hydrogens and charges using the DockPrep tool available in [UCSF Chimera](https://www.cgl.ucsf.edu/chimera/) or UCSF [ChimeraX](https://www.cgl.ucsf.edu/chimerax/)

In [1]:
#@title Installing pyLiBELa (~3 min) {display-mode: "form"}
#@markdown The instalation process requires it to reestart the Colab. Don't be afraid if it happens.

#Installing Dependencies
%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev #libopenbabel-dev openbabel
!pip install ipython-autotime
!pip install py3Dmol
!pip install shutil
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


#Compiling pyLiBELa
use_cuda = True #@param {type:"boolean"}

! rm -f Makefile*
! rm -rf obj src test
! rmdir obj src
if use_cuda:
  ! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
else:
  ! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile

! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2


time: 3min 11s (started: 2024-04-02 17:39:29 +00:00)


In [1]:
#@title Settings {display-mode: "form"}

import os
import timeit
import numpy as np

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')


Input = PARSER()
Input.output = "erick_teste" #@param {type:"string"}

Input.grid_spacing = 0.4 #@param {type:"number"}
grid_dimension = 30.0 #@param {type:"number"}

search_box = 12.0 #@param {type:"number"}

Input.atom_limit = 10000 #@param {type:"number"}



Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = False;
Input.load_grid_from_file = True;
Input.use_grids = True
Input.write_mol2 = True

Input.dielectric_model = "r"
#Input.diel = 2.0;
Input.scoring_function = 0 #@param {type:"integer"}
Input.grid_prefix = 'McGrid'
Input.solvation_alpha = 0.1
Input.solvation_beta = -0.005

# Optimization parameter:
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "ln_auglag" #@param ["subplex", "ln_auglag", "direct_only", "isres", "crs", "esch", "stogo", "mma", "none"]
Input.min_tol = 1E-5 #@param {type:'number'}
Input.min_delta = 1E-5 #@param {type:'number'}
Input.dock_min_tol = 1E-5 #@param {type:'number'}
Input.timeout = 20 #@param {type:'integer'}
Input.min_timeout = 20 #@param {type:'integer'}

if (Input.scoring_function < 3):
    delta = 2.5
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.use_score_optimizer = False #@param {type:'boolean'}

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

Input.ligand_energy_model = "MMFF94";
Input.atomic_model_ff = "AMBER";

def mol2_to_pdb(mol2file, pdbfile):
  !obabel -imol2 {mol2file} -opdb -O {pdbfile} > /dev/null

def join_pdb(name_file_rec,name_file_lig,name_file_merge):
  !cp $name_file_rec temp_rec.pdb
  !cp $name_file_lig temp_lig.pdb

  !sed -i '/END\|CONECT/d' temp_rec.pdb #tira o end e o conect do arquivo do receptor
  !sed -i 's/A  /B  /g' temp_lig.pdb #troca a cadeia A por B no arquivo do ligante
  !sed -i '/CONECT/d' temp_lig.pdb #tira o conect do arquivo do ligante
  !cat temp_rec.pdb temp_lig.pdb > $name_file_merge #concatena os arquivos
  !rm -rf temp_rec.pdb temp_lig.pdb

%cd '/content/'
!mkdir -p uploads
!mkdir -p results

Input.output = 'results/' + Input.output
Input.grid_prefix = 'results/' + Input.grid_prefix


pyLiBELa is imported!
/content


In [2]:
#@title Uploading files{display-mode: "form"}
def File_extension_manager(name):
  role_flag = False

  name_ext = name.split('.')
#  name_ext = name_ext[-1]

  if name_ext[-1] == 'mol2':

    role_flag = True
    name_ext_string = '.mol2'

  elif name_ext[-2] == 'mol2':

    role_flag = True
    name_ext_string = '.mol2.gz'

  else:
    print('Check your file extension!\nIt needs to be .mol2 or .mol2.gz.')
    name_ext_string =''

  return role_flag,name_ext_string


import ipywidgets as widgets
from IPython.display import display
from google.colab import files

%cd '/content/'
!mkdir -p uploads
%cd 'uploads'
!rm -rf *

uploaded = files.upload() #becomes a dictionary, for some reason
up_list = list(uploaded.keys())


rec_toggle = widgets.ToggleButtons(description='Receptor:',
                                   style = {'description_width': 'initial'},
                                   options=up_list)

lig_toggle = widgets.ToggleButtons(description='Reference Ligand:',
                                   style = {'description_width': 'initial'},
                                   options=up_list)

display(rec_toggle)
display(lig_toggle)

def rec_changed(changes):
  rec_name = rec_toggle.value
  #print('receptor em',rec_name)

def lig_changed(changes):
  lig_name = lig_toggle.value
  #print('ligante em',lig_name)

rec_toggle.observe(rec_changed,'value')
lig_toggle.observe(lig_changed,'value')
#print(toggle.value)

/content
/content/uploads


Saving 62wgn_rec.mol2 to 62wgn_rec.mol2
Saving 62wgn_lig.mol2 to 62wgn_lig.mol2


ToggleButtons(description='Receptor:', options=('62wgn_rec.mol2', '62wgn_lig.mol2'), style=ToggleButtonsStyle(…

ToggleButtons(description='Reference Ligand:', options=('62wgn_rec.mol2', '62wgn_lig.mol2'), style=ToggleButto…

In [3]:
#@title Preview
#@markdown Receptor in white, reference ligand in green and searchbox in blue.
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol
aminoacids = ['MET', 'THR', 'ASN', 'LYS', 'SER', 'ARG', 'VAL', 'ALA', 'ASP', 'GLU', 'GLY', 'PHE', 'LEU', 'TYR', 'CYS', 'TRP', 'PRO', 'HIS', 'GLN', 'ILE', 'ASH', 'CYX', 'GLH', 'HIE', 'HID', 'HIP','HOH']

%cd '/content/results'


rec_name = rec_toggle.value
rec_flag,rec_ext = File_extension_manager(rec_name)
print('Receptor read from file ',rec_name)

if rec_flag:
  REC = Mol2(Input, '../uploads/'+rec_name)
  print('Receptor uploaded succesfully!')
  print("It has %4d atoms." % REC.N)

print(' ')

lig_name = lig_toggle.value
lig_flag,lig_ext = File_extension_manager(lig_name)
print('Ligand read from file',lig_name)


if lig_flag:
  RefLig = Mol2(Input, '../uploads/'+lig_name)
  print('Ligand uploaded succesfully!')
  print("It has %d atoms." % RefLig.N)

!rm -rf lig_ref.pdb rec.pdb search_box.pdb
mol2_to_pdb('../uploads/' + rec_name, 'rec.pdb')
mol2_to_pdb('../uploads/' + lig_name, 'lig_ref.pdb')

lig_ref_pdb = open('lig_ref.pdb', 'r').read()
rec_pdb = open('rec.pdb','r').read()

join_pdb('rec.pdb','lig_ref.pdb','complex1.pdb')
complex1_pdb = open('complex1.pdb','r').read()

%cd /content

Coord = COORD_MC()
center = Coord.compute_com(RefLig)

xend,xbegin = center[0]+search_box/2, center[0]-search_box/2
yend,ybegin = center[1]+search_box/2, center[1]-search_box/2
zend,zbegin = center[2]+search_box/2, center[2]-search_box/2

Writer = WRITER(Input)
Writer.write_box(center, xbegin, ybegin, zbegin, xend, yend, zend)
!mv box.pdb results/search_box.pdb

search_box_pdb = open('results/search_box.pdb','r').read()
view = py3Dmol.view(width=800,height=600)

model=0

view.addModel(search_box_pdb,'pdb')
view.setStyle({'model':0},{'stick':{"color":"#4565EA","width":0.5}})

model+=1
view.addModel(lig_ref_pdb,'pdb')
view.setStyle({'model':model},{'stick': {'colorscheme':'greenCarbon'}})


model+=1
view.addModel(complex1_pdb,'pdb')
view.setStyle({'model':model,'chain':'B'},{'opacity':0})
view.setStyle({'model':model,'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})
view.setStyle({ 'model':model,'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})


view.setBackgroundColor('white')

view.zoomTo()
view.show()

/content/results
Receptor read from file  62wgn_rec.mol2
Receptor uploaded succesfully!
It has 2650 atoms.
 
Ligand read from file 62wgn_lig.mol2
Ligand uploaded succesfully!
It has 46 atoms.
1 molecule converted
1 molecule converted
/content


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:

#@title Reading search ligand {display-mode: "form"}

#Reading Ligand SMILES

!rm -rf lig2.pdb lig2.smi

ligand_smiles = "Cc1ncc(C[n+]2csc(CCOP(=O)(O)O)c2C)c(N)n1" #@param {type:"string"}

Lig2 = Mol2()
lig2_flag = False
if Lig2.parse_smiles(Input, ligand_smiles, 'Lig2'):
   print('Ligand SMILES parsed successfully!')
   print(ligand_smiles)
   lig2_flag = True

else:
  print('Oops, something wrong with you ligand... \nIt may be way too big or have an unknown atom.')

if lig2_flag:
  print('Search ligand has %4d atoms' % Lig2.N)

  f = open("results/lig2.smi", "w")
  f.write(ligand_smiles)
  f.close()

  Writer.write_pdb(Lig2, Lig2.xyz, 0.00, 0.00, 'lig2')
  !gunzip lig2.pdb.gz
  !mv lig2.pdb results/
  lig2_pdb = open('results/lig2.pdb', 'r').read()

  view = py3Dmol.view()

  view.addModel(lig2_pdb,'pdb')
  view.setStyle({'model':0},{'stick': {'colorscheme':'yellowCarbon'}})

  view.setBackgroundColor('white')

  view.zoomTo()
  view.show()

In [ ]:
#@title Grid Generation (may take several minutes if you are not using a GPU)
%load_ext autotime
!rm -rf {Input.output}_dock.mol2.gz
import timeit


Coord = COORD_MC()
HB = FindHB()
Ene  = Energy2(Input)

for i in range(len(REC.residue_pointer)-1):
  HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);

HB.find_ligandHB(lig_name, RefLig);
print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

Dock = Docker(Writer)
center = Coord.compute_com(RefLig)

print()
start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
print('Starting energy: %7.3f kcal/mol' % start_energy);
print()
print('Generating grids. This may take a while..')

start_time = timeit.default_timer()
Grids = Grid(Input, Writer, REC, center)
Input.dock_parallel = True
Input.parallel_jobs = 2
time = timeit.default_timer() - start_time

print('Grids computed, and it took %.2f s (%.2f min).' %(time,time/60.))
grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
print('Grid original energy: %7.3f kcal/mol' % grid_energy);
print('Grid error: %7.3f %%' % abs(100.*(start_energy-grid_energy)/start_energy));
print()
print()


In [ ]:
#@title Docking and showing the result

# Remove previous results, if any
! rm -f {Input.output}_dock.mol2.gz lig2.pdb

print('Starting docking calculation...')
Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
print('Docking calculation finished!')

Writer.write_box(center, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)
!mv box.pdb results/

mol2_to_pdb(Input.output+'_dock.mol2.gz', 'results/lig2.pdb')

lig2_pdb = open('results/lig2.pdb', 'r').read()
box_pdb = open('results/box.pdb','r').read()
join_pdb('results/rec.pdb','results/lig2.pdb','results/complex2.pdb')
complex2_pdb = open('results/complex2.pdb','r').read()

#Computing the RMSD with OpenBabel
!obrms uploads/{lig_name} {Input.output}_dock.mol2.gz



#@title Docked complex
#@markdown Do you want to show the grid box?
show_grid = True #@param {type:"boolean"}
#@markdown Do you want to show the search box?
show_box = False #@param {type:"boolean"}
#@markdown Do you want to show the reference ligand? If selected, it will be shown in green.
show_ref = True #@param {type:"boolean"}
#@markdown Do you want to show the target protein?
show_rec = True #@param {type:"boolean"}
#@markdown Do you want to show the search ligand? If selected, it will be shown in yellow.
show_lig2 = True #@param {type:"boolean"}
#@markdown Do you want to show residues from the protein close to the search ligand?
show_close_res = True #@param {type:"boolean"}
#@markdown How close do you want them to be? Put the radius in Angstroms
dist=5 #@param {type:"integer"}


view = py3Dmol.view(width=800,height=600)

model=0

if show_grid:
  view.addModel(box_pdb,'pdb')
  view.setStyle({'model':model}, {'stick':{"color":"#ff0000","width":2}})
  #print(model)
  model+=1

if show_box:
  view.addModel(search_box_pdb,'pdb')
  view.setStyle({'model':model},{'stick':{"color":"#4565EA","width":0.5}})
  #print(model)
  model+=1

if show_ref:
  view.addModel(lig_ref_pdb,'pdb')
  view.setStyle({'model':model},{'stick': {'colorscheme':'greenCarbon','opacity':0.7}})
  model+=1

if show_rec:

  view.addModel(complex2_pdb,'pdb')
  view.setStyle({'model':model,'chain':'B'},{'opacity':0})
  view.setStyle({'model':model,'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})
  view.setStyle({ 'model':model,'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})

  if show_close_res:
    view.addStyle({'model':model,'chain':'A','within':{'distance':'%.d'%dist, 'sel':{'chain':'B'}},'byres':True}, {'stick': {'colorscheme':'skyblueCarbon'}})

  if show_lig2:
      view.setStyle({'model':model,'chain':'B'},{'stick': {'colorscheme':'yellowCarbon'}})
  model+=1

else:
  if show_lig2:
    view.addModel(lig2_pdb,'pdb')
    view.setStyle({'model':model},{'stick': {'colorscheme':'yellowCarbon'}})
    model+=1
  if show_close_res:
    print('To show the residues close to the ligand, you need to select the show_rec box.')


view.setBackgroundColor('white')

view.zoomTo()
view.show()

In [ ]:
#@title Downloading files
import shutil
jobname = Input.output.split('/')
results = 'pyLiBELa_{}'.format(jobname[1])
!mv -r results $results
print('File {} is being prepared'.format(results))
shutil.make_archive(results, 'zip', results)
files.download('{}.zip'.format(results))